In [1]:
import pandas.io.sql
import pyodbc
import pandas as pd

In [7]:
import sys
sys.path.append('C:\\Users\\tonys\\Documents\\Research\\characterizationPaperPackage\\inst\\py\\')

In [8]:
from settings_sqlserver import *

Imported settings...
Current path:  C:\Users\tonys\Documents\Research\characterizationPaperPackage\inst\py\


In [9]:
# Create the connection
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+db+';UID='+user+';PWD='+ password)

In [10]:
import numpy as np
from tqdm import tqdm
import glob
import os

In [11]:
cohort_files = pd.read_csv('../settings/CohortsToCreateFinal.csv')

In [13]:
from time import time

In [23]:
print(type(cohort_file_id))

<class 'int'>


In [20]:
# For the finalized, 2010-2020 run
for cohort_file_id in tqdm(cohort_files.cohortId):
    print('Loading data')
    a = time()
    sql_query_string = 'select person_id, gender_concept_id, condition_concept_id, concept_name, time_to_onset from ohdsi_cumc_2021q1r2.results.sexdiff_cohort_ttonset_v5 where cohort_definition_id = '
    df = pandas.io.sql.read_sql(sql_query_string + str(cohort_file_id), conn)
    print(time() - a)
    print(cohort_file_id, len(df))
    
    # df = pd.read_csv(cohort_file, sep='|', names=names, encoding = 'unicode_escape')
    # female_df and male_df
    female_df = df[df.gender_concept_id == 8532]
    male_df   = df[df.gender_concept_id == 8507]
    
    # num_female and num_male TOTAL
    num_female_TOT = len(female_df.person_id.unique())
    num_male_TOT   = len(male_df.person_id.unique())
    
    # common_symptoms (at least one man and one woman)
    common_symptoms = set(female_df.condition_concept_id.unique()).intersection(set(male_df.condition_concept_id.unique()))
    
    data_dict = {}
    cohort_id = []
    condition = []
    condition_name = []
    num_females = []
    num_males   = []
    avg_TTD_females = []
    avg_TTD_males   = []
    std_TTD_females = []
    std_TTD_males   = []
    TOT_FEMALES_IN_COHORT = []
    TOT_MALES_IN_COHORT   = []
    
    print('Generating summary...')
    for common_condition_concept in tqdm(common_symptoms):
        cohort_id.append(cohort_file_id)
        condition.append(common_condition_concept)
        condition_name.append(female_df[female_df.condition_concept_id == common_condition_concept].concept_name.values[0])

        num_females.append(len(female_df[female_df.condition_concept_id == common_condition_concept].person_id.unique()))
        num_males.append(len(male_df[male_df.condition_concept_id == common_condition_concept].person_id.unique()))

        avg_TTD_females.append(np.mean(female_df[female_df.condition_concept_id == common_condition_concept].time_to_onset))
        std_TTD_females.append(np.std(female_df[female_df.condition_concept_id == common_condition_concept].time_to_onset))

        avg_TTD_males.append(np.mean(male_df[male_df.condition_concept_id == common_condition_concept].time_to_onset))
        std_TTD_males.append(np.std(male_df[male_df.condition_concept_id == common_condition_concept].time_to_onset))
        
        TOT_FEMALES_IN_COHORT.append(num_female_TOT)
        TOT_MALES_IN_COHORT.append(num_male_TOT)
        
    data_dict['cohort_definition_id'] = cohort_id
    data_dict['condition_concept_id'] = condition
    data_dict['concept_name'] = condition_name
    data_dict['num_females_COHORT'] = TOT_FEMALES_IN_COHORT
    data_dict['num_males_COHORT'] = TOT_MALES_IN_COHORT
    data_dict['num_females_this_symptom'] = num_females
    data_dict['num_males_this_symptom'] = num_males
    data_dict['avg_TTD_females_days'] = avg_TTD_females
    data_dict['avg_TTD_males_days'] = avg_TTD_males
    data_dict['std_TTD_females_days'] = std_TTD_females
    data_dict['std_TTD_males_days'] = std_TTD_males

    summary = pd.DataFrame.from_dict(data_dict)
    print(cohort_file_id)

  0%|                                                                                          | 0/119 [00:00<?, ?it/s]

Loading data
4.022963523864746
11111 597672
Generating summary...



 24%|██████████████████▎                                                          | 1482/6248 [00:06<00:22, 213.33it/s]


 46%|███████████████████████████████████▍                                         | 2871/6248 [00:13<00:15, 223.73it/s]


 70%|██████████████████████████████████████████████████████▏                      | 4394/6248 [00:20<00:08, 226.73it/s]


...

 94%|████████████████████████████████████████████████████████████████████████▋    | 7039/7452 [00:41<00:02, 175.84it/s]


  3%|██▋                                                                             | 4/119 [02:15<1:12:27, 37.80s/it]

11114
Loading data


  3%|██▋                                                                             | 4/119 [02:24<1:09:22, 36.19s/it]


KeyboardInterrupt: 

In [16]:
cohort_id

[11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,
 11114,


In [17]:
df.head()

,person_id,gender_concept_id,cohort_definition_id,cohort_definition_name,condition_concept_id,concept_name,cohort_start_date,condition_start_date,time_to_onset,age
0,5742252,8532,11114,[PL 31967003] Nausea Events All Events of refe...,45766714,Inflammatory dermatosis,2014-01-14,2012-03-02,683,15.0
1,146066,8507,11114,[PL 31967003] Nausea Events All Events of refe...,45766714,Inflammatory dermatosis,2015-08-05,2015-02-18,168,84.0
2,8611007,8532,11114,[PL 31967003] Nausea Events All Events of refe...,45766714,Inflammatory dermatosis,2019-04-15,2018-08-20,238,92.0
3,7415410,8532,11114,[PL 31967003] Nausea Events All Events of refe...,45766714,Inflammatory dermatosis,2015-05-06,2013-07-30,645,8.0
4,1113108,8532,11114,[PL 31967003] Nausea Events All Events of refe...,45766714,Inflammatory dermatosis,2014-11-04,2014-10-10,25,81.0


In [18]:
a = time()
sql_query_string = 'select * from ohdsi_cumc_2021q1r2.results.sexdiff_cohort_ttonset_v5 where cohort_definition_id = '
df = pandas.io.sql.read_sql(sql_query_string + str(cohort_file_id), conn)
print(time() - a)

16.52379059791565


In [19]:
# Four times faster select statement, but is this enough?
b = time()
sql_query_string = 'select person_id, gender_concept_id, condition_concept_id, concept_name, time_to_onset from ohdsi_cumc_2021q1r2.results.sexdiff_cohort_ttonset_v5 where cohort_definition_id = '
df = pandas.io.sql.read_sql(sql_query_string + str(cohort_file_id), conn)
print(time() - b)

4.794788360595703


In [21]:
summary.head()

,cohort_definition_id,condition_concept_id,concept_name,num_females_COHORT,num_males_COHORT,num_females_this_symptom,num_males_this_symptom,avg_TTD_females_days,avg_TTD_males_days,std_TTD_females_days,std_TTD_males_days
0,11114,196620,Viral enteritis,34134,15237,29,24,189.413793,362.375000,295.968721,354.724636
1,11114,196625,Viral hepatitis A without hepatic coma,34134,15237,13,9,397.076923,271.666667,447.309128,262.448302
2,11114,196653,Malignant tumor of kidney,34134,15237,20,32,218.850000,357.812500,311.044575,379.138533
3,11114,196654,Lipoma of intra-abdominal organs,34134,15237,12,6,387.666667,563.666667,326.175396,287.396165
4,11114,4128827,Voice production finding,34134,15237,38,15,469.342105,587.200000,350.824688,296.031350
